In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df['Exited'].value_counts(), df['Geography'].value_counts(), df['Gender'].value_counts()

df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)

In [4]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [7]:
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [9]:
class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.fc1 = nn.Linear(11, 6)
        self.fc2 = nn.Linear(6, 6)
        self.fc3 = nn.Linear(6, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [10]:
model = ChurnModel()

In [11]:
from torchsummary import summary

summary(model, input_size=(11,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 6]              72
            Linear-2                    [-1, 6]              42
            Linear-3                    [-1, 1]               7
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [12]:
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [14]:
EPOCHS = 100

X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train.values).float().view(-1, 1)

for epoch in range(EPOCHS):
    optimizer.zero_grad()
    y_pred = model(X_train_tensor)
    l = loss(y_pred, y_train_tensor)
    l.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'epoch {epoch}: {l.item()}')

epoch 0: 0.7695948481559753
epoch 10: 0.6438363790512085
epoch 20: 0.545135498046875
epoch 30: 0.4750314950942993
epoch 40: 0.44014737010002136
epoch 50: 0.4263213872909546
epoch 60: 0.41687726974487305
epoch 70: 0.4112318754196167
epoch 80: 0.40605878829956055
epoch 90: 0.4010094106197357


In [16]:
with torch.no_grad():
    y_pred = model(torch.tensor(X_test.values))
    correct = (torch.round(y_pred).squeeze() == torch.tensor(y_test.values)).sum().item()

    print(f'Accuracy: {correct/len(y_test)}')

Accuracy: 0.7975


In [18]:
from sklearn.metrics import accuracy_score

# Ensure y_test is a tensor and on the same device as y_pred
y_test_tensor = torch.tensor(y_test.values, device=y_pred.device)
accuracy = accuracy_score(y_test_tensor.cpu(), torch.round(y_pred).cpu())

In [19]:
accuracy

0.7975